# Just another EDA notebook
I'm just sharing the basic EDA work that I did before looking at other folks' notebooks. Give it an upvote if you found it useful...

# Contest overview
## Objective
* Show customers products they'll be more likely to buy. 
* Make 12 predictions, ranked 1-12 for each customer, higher ranked that customer buys = better score.

## Data
Basic observations on looking at the data in the explorer
* Images
    * Folders are not organized to anything useful, simply by three digits of item id
    * file id= article id
* Articles.csv = 25 columns x 109m rows, describing the items for sale
* customers.csv = 7 columns 1.4m rows
    * Age, member status, fashion news, 
    * postal code - hash value one value with 9%, seems strange
* transcactions_train.csv
    * One line per customer/item bought... if mutliple items bought then mutliple lines
    * Price column... min = 0, max = 0.59, mean = 0.03... is this a % of price paid?
    * sale_channel_id = either 1 or 2... not sure what this means






In [ ]:
import os
import pandas as pd
pd.options.plotting.backend = "plotly"
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [ ]:
%%time
#takes ~90 seconds without accelerator
path = '../input/h-and-m-personalized-fashion-recommendations/'

articles_df = pd.read_csv(path + 'articles.csv')
customers_df = pd.read_csv(path + 'customers.csv')
sample_submission_df =  pd.read_csv(path + 'sample_submission.csv')
transactions_train_df =  pd.read_csv(path + 'transactions_train.csv')

images_jpg = glob(path + "images/*/*.jpg")
f'df shapes: articles{articles_df.shape}, customers{customers_df.shape}, transactions{transactions_train_df.shape}, no of images: {len(images_jpg)}, submissions{sample_submission_df.shape}'

In [ ]:
articles_df.describe()

In [ ]:
articles_df.tail()

In [ ]:
customers_df.tail()

In [ ]:
transactions_train_df.head()

# Distributions
I'm curious to see:
* Total items:
 * What are the most popular items?
 * What are the most popular items by customers in postal codes/ages/etc?
* Items over time:
 * Total items sold in a trend line over time, broken down by various attributes of the items
 * Should we be predicting certian items at certian times of the year?

In [ ]:
transactions_count_df = pd.DataFrame.from_dict(transactions_train_df['article_id'].value_counts()) #creates a dictionary of counts of items as a df
transactions_count_df.reset_index(level=0, inplace=True) #want index to be standard pd row numbers, not the item id...
transactions_count_df.rename(columns={"index":'article_id','article_id':'count'}, inplace=True)
transactions_count_df.tail()

In [ ]:
transactions_count_df['index1'] = transactions_count_df.index #don't want to index on count
transactions_count_df.drop(['index1'], axis=1, inplace= True) 
transactions_count_df.head()

In [ ]:
#Lots of zeros or ones...
transactions_count_df['count'].plot()

In [ ]:
#df_temp.plot()
transactions_count_df[transactions_count_df < 1000]['count'].plot()

In [ ]:
# all sales by date
transactions_train_df.tail()

In [ ]:
articles_df.columns

In [ ]:
# eyeballing some columns to merge with transactions
article_cols = ['article_id', 'product_type_name', 'product_group_name', 'department_no', 'department_name', 'index_code', 'index_group_no', 
                'index_group_name', 'section_no', 'section_name', 'garment_group_no', 'garment_group_name']

articles_df[article_cols].tail()

In [ ]:
transactions_with_ref_df = pd.merge(transactions_train_df, articles_df[article_cols], on= 'article_id', how='left')
transactions_with_ref_df.tail()

In [ ]:
# group new df by weeks... see seasonal trends... y = number of transactions, x = week
# stacked bar chart by garment group name (seems reasonable to try first)
# 